### ChromaDB에 벡터 저장

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# PDF 파일 로드 및 분할
loader = PyPDFLoader("data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

# 청크 크기 및 중첩 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100)
docs = text_splitter.split_documents(pages)

# HuggingfaceEmbedding 함수로 Open source 임베딩 모델 로드
model_name = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
ko_embedding = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

# ChromaDB에 저장하기 위한 인스턴스 생성
vectorstore = Chroma(
    collection_name = "full_documents",
    embedding_function = ko_embedding
)

/opt/anaconda3/envs/RAG_CODE/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### MutliVectorRetriever 생성
사용자의 질문과 유사한 문서를 찾을 때 하위 청크를 검색하고 이를 기반으로 상위 청크를 호출하여 LLM에게 더 완전한 맥락의 정보를 전달
- retriever.vectorstore의 similarity_search() 함수 실행 결과는 하위 청크 검색 결과를 반환
- retriever의 get_relevant_documents() 함수 실행 결과는 상위 청크 검색 결과를 반환
- retriever.vectorstore라는 벡터 DB 기반으로 하위 유사 청크를 검색, retriever에서는 해당 청크들의 문서 ID를 타고 올라가 상위 청크를 반환

In [2]:
from langchain.storage import InMemoryByteStore                    # 상위 청크와 하위 청크를 연결하여 저장할 저장소 생성
from langchain.retrievers.multi_vector import MultiVectorRetriever # 상위 문서 검색기 생성
import uuid                                                        # 문서 ID로 고유한 값을 지정하기 위해 uuid 라이브러리 호출

# 상위 문서 저장을 위한 레이어 선언
store = InMemoryByteStore()
id_key = "doc_id"

# 상위 문서와 하위 문서를 연결할 키값으로 doc_id 사용
retriever = MultiVectorRetriever(
    vectorstore = vectorstore,
    byte_store = store,
    id_key = id_key,
)

# 문서 ID 생성
doc_ids = [str(uuid.uuid4()) for _ in docs]

# 하위 청크로 쪼개기 위한 child_text_splitter 지정
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap = 50)

# 상위 청크들을 순회하며 하위 청크로 분할한 후 상위 청크 id 상속
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)

# vectorstore에 하위 청크 추가
retriever.vectorstore.add_documents(sub_docs)

# docstore에 상위청크 저장할 때, doc_ids 지정
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [3]:
print("[하위 청크] \n")
print(retriever.vectorstore.similarity_search("국민의 권리")[0].page_content)
print("-"*50)
print("[상위 청크] \n")
print(retriever.invoke("국민의 권리")[0].page_content)

[하위 청크] 

법제처                                                            2                                                       국가법령정보센터
대한민국헌법
 
       제2장 국민의 권리와 의무
 
제10조 모든 국민은 인간으로서의 존엄과 가치를 가지며, 행복을 추구할 권리를 가진다. 국가는 개인이 가지는 불가침
의 기본적 인권을 확인하고 이를 보장할 의무를 진다.
--------------------------------------------------
[상위 청크] 

법제처                                                            2                                                       국가법령정보센터
대한민국헌법
 
       제2장 국민의 권리와 의무
 
제10조 모든 국민은 인간으로서의 존엄과 가치를 가지며, 행복을 추구할 권리를 가진다. 국가는 개인이 가지는 불가침
의 기본적 인권을 확인하고 이를 보장할 의무를 진다.
 
제11조 ①모든 국민은 법 앞에 평등하다. 누구든지 성별ㆍ종교 또는 사회적 신분에 의하여 정치적ㆍ경제적ㆍ사회적ㆍ
문화적 생활의 모든 영역에 있어서 차별을 받지 아니한다.
②사회적 특수계급의 제도는 인정되지 아니하며, 어떠한 형태로도 이를 창설할 수 없다.
③훈장등의 영전은 이를 받은 자에게만 효력이 있고, 어떠한 특권도 이에 따르지 아니한다.
